In [76]:
! pip install vegas


Tutorial
=======================================

Run through the tutorial at https://vegas.readthedocs.io/en/latest/tutorial.html. Some of the code is copied here already.

.. moduleauthor:: G. Peter Lepage <g.p.lepage@cornell.edu>

.. |Integrator| replace:: :class:`vegas.Integrator`
.. |PDFIntegrator| replace:: :class:`vegas.PDFIntegrator`
.. |AdaptiveMap| replace:: :class:`vegas.AdaptiveMap`
.. |vegas| replace:: :mod:`vegas`
.. |WAvg| replace:: :class:`vegas.RunningWAvg`
.. |chi2| replace:: :math:`\chi^2`
.. |x| replace:: x
.. |y| replace:: y
.. |~| unicode:: U+00A0
.. |times| unicode:: U+00D7
   :trim:

Introduction
-------------

Class :class:`vegas.Integrator` gives Monte Carlo estimates of arbitrary
multidimensional integrals using the *vegas* algorithm
(G. P. Lepage, J. Comput. Phys. 27 (1978) 192 and J. Comput. Phys. 439 (2021) 110386).
The algorithm has two components.
First an automatic transformation is applied to to the integration variables
in an attempt to flatten the integrand. Then a Monte Carlo estimate of the
integral is made using the  transformed variables. Flattening the integrand
makes the integral easier and improves the estimate.  The transformation
applied to the integration variables is optimized
over several iterations of the algorithm: information about the integrand that
is collected during one iteration is used to  improve the transformation used
in the next iteration.

Monte Carlo integration makes few assumptions about the
integrand --- it needn't be analytic nor even continuous. This
makes Monte Carlo integration unusually robust. It also makes it well suited
for adaptive integration. Adaptive strategies are essential for
multidimensional integration, especially in high dimensions, because
multidimensional space is large, with  lots of corners, making it
easy to lose important features in the integrand.

Monte Carlo integration also provides efficient and reliable methods for
estimating the
accuracy of its results. In particular, each Monte Carlo
estimate of an integral is a random number from a distribution
whose mean is the correct value of the integral. This distribution is
Gaussian or normal provided
the number of integrand samples is sufficiently large.
In practice we generate multiple
estimates of the integral
in order to verify that the distribution is indeed Gaussian.
Error analysis is straightforward if the
integral estimates are Gaussian.

The |vegas| algorithm has been in use for decades and implementations are
available in many programming languages, including Fortran (the original
version), C and C++. The algorithm used here is significantly improved over
the original implementation, and that used in most other implementations.
It uses two adaptive strategies: importance sampling, as in the original
implementation, and adaptive stratified sampling, which is new. The 
new algorithm is described in G. P. Lepage, arXiv_2009.05112_ 
(J. Comput. Phys. 439 (2021) 110386).

.. _arXiv_2009.05112: https://arxiv.org/abs/2009.05112

There is also a third adaptive strategy, adaptive re-stratification, that can 
be useful in very high dimensions. See :func:`vegas.restratify`.

This module is written in Cython, so it is almost as fast as compiled Fortran or
C, particularly when the integrand is also coded in Cython (or some other
compiled language), as discussed below.

The following sections describe how to use |vegas|. Almost every
example shown is a complete code, which can be copied into a file
and run with Python. It is worthwhile playing with the parameters to see how
things change.


Basic Integrals
----------------
Here we illustrate the use of |vegas| by estimating the integral

$$
    C\int_{-1}^1 dx_0 \int_0^1 dx_1 \int_0^1 dx_2 \int_0^1 dx_3
    \,\,\mathrm{e}^{- 100 \sum_{d}(x_d-0.5)^2}  ,
$$
where constant $C$ is chosen so that the exact integral is 1.
The following code shows how this can be done::


In [77]:
import vegas
import math
import numpy as np
from scipy.special import gamma

In [96]:
def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return math.exp(-dx2 * 100.) * 1013.2118364296088

integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

result = integ(f, nitn=10, neval=10000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))


itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.29(19)        1.29(19)            0.00     1.00
  2   0.809(41)       0.831(40)           6.22     0.01
  3   0.976(24)       0.938(21)           7.96     0.00
  4   1.013(11)       0.9969(95)          8.77     0.00
  5   1.0067(88)      1.0022(65)          6.72     0.00
  6   0.9950(73)      0.9990(48)          5.49     0.00
  7   0.9861(56)      0.9935(37)          5.07     0.00
  8   0.9962(50)      0.9945(30)          4.38     0.00
  9   1.0000(43)      0.9962(24)          3.97     0.00
 10   1.0015(37)      0.9978(20)          3.68     0.00

result = 0.9978(20)    Q = 0.00


### Larger number of dimensions
Let's try a similar integral, but up the number of dimensions, $N$. In the following, we implement a Gaussian function in `n_dim` dimensions. The function is normalized, so if we integrate a sufficient number of "sigmas", the integral should just be 1. 

With the default parameters given below, the algorithms seems to work well up to around 10 dimensions. For $10\lesssim N \lesssim 30$, the integral still converges to 1, but the $Q$ parameter goes to 0 (estimate of reliability), and the running average is unreliable. For larger values, the integral does not converge well.  

In [100]:
n_dim = 10
sigma = 1.0
mean = np.zeros(n_dim)
def f(x):
    return np.exp(-0.5 * np.sum((x - mean)**2) / sigma**2) / (2 * math.pi * sigma**2)**(n_dim/2)

V = np.array([[-3 * sigma, 3 * sigma]] * n_dim) # Integrate out to +/-5 sigma
print(V.shape)

integ = vegas.Integrator(V)

result = integ(f, nitn=100, neval=1000)
print(result.summary())
print(f"result = {result} \t Q = {result.Q}")


(10, 2)
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.96(26)        0.96(26)            0.00     1.00
  2   0.82(27)        0.89(19)            0.16     0.69
  3   0.85(18)        0.87(13)            0.09     0.91
  4   0.95(10)        0.918(80)           0.13     0.94
  5   0.93(10)        0.923(63)           0.10     0.98
  6   0.912(58)       0.917(43)           0.09     0.99
  7   0.988(48)       0.948(32)           0.28     0.95
  8   0.884(29)       0.913(21)           0.55     0.80
  9   0.956(28)       0.929(17)           0.67     0.71
 10   0.970(21)       0.945(13)           0.84     0.58
 11   0.984(17)       0.960(10)           1.08     0.37
 12   0.955(14)       0.9579(83)          0.99     0.45
 13   0.977(12)       0.9643(68)          1.06     0.39
 14   0.972(11)       0.9665(57)          1.01     0.44
 15   0.9888(90)      0.9729(48)          1.24     0.24
 16   0.9732(85)      0.9730(42)        

### Tricky/weird function: a "shell"
Consider a shell of radius $r$ in $N$ dimensions. If we take a random variable uniformly distributed on the surface of the shell, and project it onto a single coordinate, the resulting probability distribution is:

$$
p_N(x) = \frac{\Gamma(\frac{n}{2}+1)}{\sqrt{\pi} R \Gamma(\frac{n+1}{2})} \left(1 - \frac{x^2}{R^2}\right)^{\frac{n-3}{2}}
$$

This is an example of a distribution where 1D projections don't reflect the ND distribution very well. So, we might expect VEGAS to have some problems with a "shell" function, defined like:

$$
f(x; R, dR) = \left\{ \begin{matrix} 1 & : & |(||x|| - R)| < dR \\ 0 & : & \text{otherwise} \end{matrix}\right.
$$

For example, in 2 dimensions (a circle):

$$
p_2(x) = \frac{1}{\pi\sqrt{R^2-x^2}}
$$

This diverges at $x = \pm R$ (where the "walls of the circle are vertical"), so we might expect some bad behavior from the algorithm here.

In $N=3$ dimensions, 

$$
p_3(x) = \frac{1}{2R}
$$

This is a uniform distribution, so VEGAS will probably perform similarly to basic MC integration without importance sampling.

In [104]:
n_dim = 2
height = 1
r0 = 1.0 # radius of shell
dr = 0.01 # half-thickness of shell

# Expected integral
def n_sphere_area(r, n):
    return 2. * math.pi**(n/2) / gamma(n/2)
#print(f"Unit circle diameter = {n_sphere_area(1, 2)/math.pi}*pi")
#print(f"Unit sphere area = {n_sphere_area(1, 3)/math.pi}*pi")
print(f"Expected integral = {height * 2*dr * n_sphere_area(r0, n_dim)}")

def f(x):
    xarr = np.asarray(x)
    r = np.sum(xarr * xarr)**0.5
    if np.abs(r - r0) < dr:
        return height
    else:
        return 0

# Integrate over N-cube [-2, 2]^N
V = np.array([[-2 * r0, 2 * r0]] * n_dim)
print(V.shape)

integ = vegas.Integrator(V)

result = integ(f, nitn=1000, neval=10000)
print(result.summary())
print(f"result = {result} \t Q = {result.Q}")


Expected integral = 0.12566370614359174
(2, 2)
 itn   integral        wgt average     chi2/dof        Q
--------------------------------------------------------
   1   0.134(14)       0.134(14)           0.00     1.00
   2   0.089(16)       0.114(10)           4.76     0.03
   3   0.25(16)        0.115(10)           2.73     0.07
   4   0.145(42)       0.116(10)           1.98     0.11
   5   0.0566(81)      0.0803(63)          6.96     0.00
   6   0.077(19)       0.0800(60)          5.57     0.00
   7   0.057(13)       0.0760(54)          5.08     0.00
   8   0.0476(44)      0.0588(34)          6.72     0.00
   9   0.0665(84)      0.0599(32)          5.97     0.00
  10   0.0629(74)      0.0604(29)          5.32     0.00
  11   0.0681(83)      0.0612(27)          4.87     0.00
  12   0.0655(97)      0.0615(26)          4.44     0.00
  13   0.080(17)       0.0620(26)          4.16     0.00
  14   0.107(19)       0.0628(26)          4.24     0.00
  15   0.107(29)       0.0631(26)        